In [ ]:
import os
import sys
import numpy as np
import tensorflow as tf
import tensorflow.math as tfm
import tensorflow_probability as tfp
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

current_dir = %pwd
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

from datagen import make_breast_cancer, make_digit, make_mnist, covertype

In [24]:
def plot_calibration(y_true, prob_pred, label, title, n_bins=10):

    # True and Predicted Probabilities
    true_pos, pred_pos = calibration_curve(y_true, prob_pred, n_bins=n_bins)
 
    # Plot the Probabilities Calibrated curve
    plt.plot(pred_pos, true_pos, marker='o', linewidth=1, label=label)
 
    # Plot the Perfectly Calibrated by Adding the 45-degree line to the plot
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')
 
    # Set the title and axis labels for the plot
    plt.title('Probability Calibration Curve (' + title + ')')
    plt.xlabel('Predicted Probability')
    plt.ylabel('True Probability')
 
    # Add a legend to the plot
    plt.legend(loc='best')
    plt.show()

In [25]:
def calculate_ece(y_true, y_prob, n_bins=10):
    bin_bounds = np.linspace(0, 1, n_bins + 1)
    bin_lowers = bin_bounds[:-1]
    bin_uppers = bin_bounds[1:]
    
    ece = 0.0
    for bin_lower, bin_upper in zip(bin_lowers, bin_uppers):
        # Indices of probabilities in this bin
        in_bin = np.where((y_prob > bin_lower) & (y_prob <= bin_upper))[0]
        if len(in_bin) > 0:
            # Actual accuracy in this bin
            bin_accuracy = np.mean(y_true[in_bin] == 1)
            # Average predicted probability in this bin
            bin_score = np.mean(y_prob[in_bin])
            # Weight by the number of samples in the bin
            bin_weight = len(in_bin) / len(y_prob)
            # Accumulate weighted absolute difference
            ece += np.abs(bin_score - bin_accuracy) * bin_weight

    return ece

In [119]:
rseed = 42
tr = 1000
ft = 20
dat = "cov1"

In [ ]:
if dat == "wbc":
    input_shape, trn_ds, train_ds, val_ds, test_ds = make_breast_cancer(rseed)
elif dat == "digit":
    input_shape, trn_ds, train_ds, val_ds, test_ds = make_digit(rseed)
elif dat == "mnist":
    input_shape, trn_ds, test_ds = make_mnist(1000, rseed)
elif dat == "cov1":
    input_shape, trn_ds, test_ds = covertype(1, 1000, rseed)
elif dat == "cov2":
    input_shape, trn_ds, test_ds = covertype(2, 1000, rseed)

y_true = []
for input, target in test_ds:
    y_true.append(target.numpy())
y_true = np.concatenate(y_true).reshape(-1)

In [ ]:
res_nn = np.load(f"your_nn_res_path.npz", allow_pickle=True)
prob_nn = res_nn['proba']
ece_nn = calculate_ece(y_true, prob_nn)
print("NN ECE: %f " % ece_nn)
true_pos, pred_pos = calibration_curve(y_true, prob_nn, n_bins=10)
plt.figure(figsize=(6,6))
plt.plot(pred_pos, true_pos, marker='o', linewidth=1, label='NN')

Ls = [-1, 10, 1000]
for L in Ls:
    res_hmc = np.load(f"your_res_path/res_{dat}_{tr}_-4_hmc_{L}_{ft}_-4.npz", allow_pickle=True)
    prob_hmc = res_hmc['prob']
    prob_hmc = np.concatenate(prob_hmc).reshape(-1)

    # ECE
    ece_hmc = calculate_ece(y_true, prob_hmc)
    if L == -1:
        print(f"Gibbs ECE: {ece_hmc}")
    else:
        print(f"HMC_{L} ECE: {ece_hmc}")
    
    # True and Predicted Probabilities
    true_pos, pred_pos = calibration_curve(y_true, prob_hmc, n_bins=10)
 
    # Plot the Probabilities Calibrated curve
    label = 'Gibbs' if L == -1 else f'HMC(L={L})'
    plt.plot(pred_pos, true_pos, marker='o', linewidth=1, label=label)

# Plot the Perfectly Calibrated by Adding the 45-degree line to the plot
plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')

# Add a legend to the plot
plt.legend(loc='best')
plt.title(f"Calibration Curve (Covertype)")
plt.xlabel("Mean Predicted Probabilities (Positive class:1)")
plt.ylabel("Fraction of positives (Positive class:1)")
plt.show()